# Imports and set up

WARNING: THIS NOTEBOOK IS QUITE ARM CONSUMING, SINCE IT LOADS ALL THE DATA INTO PANDAS!

In [1]:
import pandas as pd
import numpy as np

import os
import glob

import logging
from datetime import datetime, date

from tqdm import tqdm

In [2]:
WORKING_DIR = '/home/projects/genomics'
DATA_DIR = '/home/projects/genomics/active_sites/correct'
CONCATED_DF_PATH = '/home/projects/genomics/active_sites_mapped/active_sites_concated.csv'
FILES_DF_PATH = '/home/projects/genomics/active_sites_mapped/active_sites_id_index.csv'
LOGGING_PATH = f'/home/projects/genomics/active_sites_mapped/mapping-{date.today()}.log'

In [3]:
logging.basicConfig(filename=LOGGING_PATH, level=logging.DEBUG)

In [4]:
started = datetime.now()
logging.getLogger().addHandler(logging.StreamHandler())

logging.info(F'STARTED AT: {started}')

STARTED AT: 2022-08-16 18:43:06.351306


# Get the data

WARNING: I will get and concat all the data files later, after filtering `files_df`

### Dataframe of files

In [5]:
filenames = [os.path.basename(x) for x in glob.glob(DATA_DIR + '/*')]
files_df = pd.DataFrame(filenames, columns=['file_name'])
logging.info('Imported filenames to pandas')


Imported filenames to padnas


# Extract info from filenames

In [6]:
files_df['ensp_id'] = files_df.apply(lambda row: row['file_name'].split('.')[0], axis='columns')

In [7]:
def get_ensp_version(row):
    try:
        right_part = row['file_name'].split('.')[1]
    except IndexError as e:
        logging.warning(f'{row["file_name"]} has wrong filename format, skipping...')
        logging.error(e)
        logging.warning(f'{row["file_name"]}: no ensp_version')
        return np.nan
    return right_part.split('_')[0]

In [8]:
files_df['ensp_version'] = files_df.apply(get_ensp_version, axis='columns')

In [9]:
def get_pdb_id(row):
    try:
        right_part = row['file_name'].split('.')[1]
    except IndexError as e:
        logging.warning(f'{row["file_name"]} has wrong filename format, skipping')
        logging.error(e)
        logging.warning(f'{row["file_name"]}: no pdb_id')
        return np.nan
    return right_part.split('_')[1]

In [10]:
files_df['pdb_id'] = files_df.apply(get_pdb_id, axis='columns')

In [11]:
def get_chain(row):
    try:
        right_part = row['file_name'].split('.')[1]
    except IndexError as e:
        logging.warning(f'{row["file_name"]} has wrong filename format, skipping')
        logging.error(e)
        logging.warning(f'{row["file_name"]}: no chain')
        return np.nan
    return right_part.split('_')[2]

In [12]:
files_df['chain'] = files_df.apply(get_chain, axis='columns')

In [13]:
def get_ligand(row):
    try:
        right_part = row['file_name'].split('.')[1]
    except IndexError as e:
        logging.warning(f'{row["file_name"]} has wrong filename format, skipping')
        logging.error(e)
        logging.warning(f'{row["file_name"]}: no ligand')
        return np.nan
    return right_part.split('_')[3]

In [14]:
files_df['ligand'] = files_df.apply(get_ligand, axis='columns')

In [15]:
logging.info('Extracted ensp_id, ensp_version, pdb_id, chain, ligand from filenames')

Extracted ensp_id, ensp_version, pdb_id, chain, ligand from filenames


# Drop duplicated structures

For now we are not taking chain int oaccount:

In [16]:
selected_structures_df = files_df.drop_duplicates(subset=['ensp_id', 'chain', 'ligand'])

In [17]:
selected_structures_df.to_csv(FILES_DF_PATH)
logging.info(f'Saved selected_structures_df to {FILES_DF_PATH}')

Saved selected_structures_df to /home/projects/genomics/active_sites_mapped/active_sites_id_index.csv


# Concat files with AA positions

In [18]:
def read_tsv_dir(dir_name):
    """
    Reads all tsv files in a directory and returns a list of pandas dataframes (filenames are keys),
    perfoming some filtering and adding `file_name` column.
    """
    # TODO extensions etc

    df_list = []
    filenames = glob.glob(dir_name + '/*')
    columns = ['position_ensp', 'aa_ensp', 'position_pdb', 'aa_pdb']

    for path, _ in zip(filenames, tqdm(range(len(filenames)))):
        file = os.path.basename(path)
        df = pd.read_csv(path, sep='\t', names=columns)
        df['file_name'] = file
        df_list.append(df)
    
    return df_list

###  Import data files to pandas

In [19]:
df_list = read_tsv_dir(DATA_DIR)
logging.info('Loaded all tsv files into list of pandas dataframes')

100%|█████████▉| 459115/459116 [1:18:17<00:00, 97.74it/s] 
Loaded all tsv files into list of pandas dataframes


In [20]:
concated_df = pd.concat(df_list)
logging.info('Concated all pandas dataframes')

Concated all pandas dataframes


### Filter using selected structures

In [21]:
filtered_concated_df = concated_df.query('file_name in @selected_structures_df.file_name.values')


### Save to csv

In [22]:
filtered_concated_df.to_csv(CONCATED_DF_PATH)
logging.info(f'Saved filtered_concated_df to {CONCATED_DF_PATH}')

Saved filtered_concated_df to /home/projects/genomics/active_sites_mapped/active_sites_concated.csv


In [23]:
logging.info('Paths can be found in the first section of the notebook')

finished = datetime.now()
logging.info(f'FINISHED AT: {finished}')

Paths can be found in the first section of the notebook
FINISHED AT: 2022-08-16 20:03:02.421967
